In [2]:
import transformers
from transformers import EvollaModel, EvollaConfig, EvollaForProteinText2Text
# model_feature = EvollaModel.from_pretrained("/zhouxibin/workspaces/transformers/evolla-base")
hf_model = transformers.EvollaForProteinText2Text.from_pretrained("/zhouxibin/workspaces/transformers/evolla-base")
hf_model = hf_model.to("cuda")

hf_processor = transformers.EvollaProcessor.from_pretrained("/zhouxibin/workspaces/transformers/evolla-base")


Loading checkpoint shards: 100%|██████████| 9/9 [00:12<00:00,  1.39s/it]
Some kwargs in processor config are unused and will not have any effect: protein_max_length, text_max_length. 


In [ ]:
import sys
sys.path.append('/zhouxibin/workspaces/Evolla/')
from utils.others import load_config, load_model_from_config
config_path = "/zhouxibin/workspaces/Evolla/config/Evolla_10B.yaml"
config = load_config(config_path)
model = load_model_from_config(config, local_rank=0, dtype="bf16")

/root/miniconda3/envs/pl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Generate config is provided, the generate config is:  {'max_new_tokens': 512, 'do_sample': True, 'temperature': 0.6, 'top_p': 0.9}


Some weights of EsmForMaskedLM were not initialized from the model checkpoint at /zhouxibin/models/huggingface/SaProt_650M_AF2 and are newly initialized: ['esm.contact_head.regression.bias', 'esm.contact_head.regression.weight', 'esm.embeddings.position_embeddings.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Quantization is disabled


Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.65s/it]


In [12]:
import transformers
import torch
import pickle as pkl
uniprot_id = "C9RH78"
aa_seq = "MLLEETLKSCPIVKRGKYHYFIHPISDGVPLVEPKLLREVATRIIKIGNFEGVNKIVTAEAMGIPLVTTLSLYTDIPYVIMRKREYKLPGEVPVFQSTGYSKGQLYLNGIEKGDKVIIIDDVISTGGTMIAIINALERAGAEIKDIICVIERGDGKKIVEEKTGYKIKTLVKIDVVDGEVVIL"
foldseek = "dvvvvqqqpfawdddppdtdgcgclapvpdpddpvvlvvllvlcvvpadpvqaqeeeeeddscpsnvvsncvvpvhyydywylddppdppkdwqwf######gitidpdqaaaheyeyeeaeqdqlrvvlsvvvrcvvrnyhhrayeyaeyhycnqvvccvvpvghyhynwywdqdpsgidtd"
question = "What is the catalytic activity of this protein?"
hf_inputs = hf_processor(proteins=[{"aa_seq": aa_seq, "foldseek": foldseek}], messages_list=[[{"role": "system", "content": "You are an AI expert that can answer any questions about protein."}, {"role": "user", "content": question}]]).to("cuda")
with torch.no_grad():
    hf_output = hf_model.forward(**hf_inputs, output_hidden_states=True)
hf_output.hidden_states = [h.to("cpu") for h in hf_output.hidden_states]
hf_output.logits = hf_output.logits.to("cpu")
hf_output
with open("hf_hidden_states.pkl", "wb") as f:
    pkl.dump(hf_output.hidden_states, f)
with open("hf_logits.pkl", "wb") as f:
    pkl.dump(hf_output.logits, f)
# mixed_sequence = "".join([a + f for a, f in zip(aa_seq, foldseek)])
# outputs = model.forward(seqs=[mixed_sequence], questions=[question], foldseeks=None, answers=None)
# outputs

# save ckpt to huggingface format

In [ ]:
# state_dict_path = "/yuanfajie/ProteinQA/weights/llama3/227_mix_10m_swissprot_zk_data_protrek_12_step=310000-global_datasample=76799744-valid_loss_key=0.59092/epoch=7-step=310000-global_datasample=76799744-valid_loss_key=0.59092.ckpt/checkpoint/mp_rank_00_model_states.pt"
state_dict_path = "/yuanfajie/zyq-ProteinQA/DPO_ckpt_from_227_step_310000/step=63240-global_datasample=2023680-valid_loss=4.819.ckpt/checkpoint/mp_rank_00_model_states.pt"
import torch
state_dict = torch.load(state_dict_path)
print(state_dict.keys())

dict_keys(['module', 'buffer_names', 'optimizer', 'param_shapes', 'frozen_param_shapes', 'shared_params', 'frozen_param_fragments', 'lr_scheduler', 'data_sampler', 'random_ltd', 'sparse_tensor_module_names', 'skipped_steps', 'global_steps', 'global_samples', 'dp_world_size', 'mp_world_size', 'ds_config', 'ds_version', 'epoch', 'global_step', 'pytorch-lightning_version', 'loops', 'callbacks', 'lr_schedulers'])


In [4]:
model_state_dict = model.state_dict()
ckpt_state_dict = state_dict['module']
len(model_state_dict), len(ckpt_state_dict)

1055

In [4]:
mapping_dict = {
    "llm.model.model": "model.llm",
    "llm.model.lm_head": "lm_head",
    # "llm.model.model.layers": "llm.layers"
    "protein_encoder.model.esm.contact_head": None,
    "protein_encoder.model.esm": "model.protein_encoder.model",
    "protein_encoder.model.lm_head": None,
    "protein_encoder.resampler": "model.protein_encoder.sequence_compressor_resampler"
}

applied_ckpt_state_dict = {}
for ckpt_k in ckpt_state_dict.keys():
    # Check if any key in mapping_dict is a prefix of ckpt_k
    matched = False
    for mapping_key in mapping_dict.keys():
        if ckpt_k.startswith(mapping_key):
            # Use the mapping for the prefix
            model_k_start = mapping_dict[mapping_key]
            if model_k_start is None:
                print(f"Skipping: {ckpt_k}")
                matched = True
                break
            model_k = ckpt_k.replace(mapping_key, model_k_start)
            try:
                assert ckpt_state_dict[ckpt_k].shape == model_state_dict[model_k].shape, f"Shape mismatch: {ckpt_k} -> {model_k}"
            except KeyError:
                print(ckpt_k, model_k, mapping_key)
            applied_ckpt_state_dict[model_k] = ckpt_state_dict[ckpt_k]
            matched = True
            # print(f"Matched: {ckpt_k} -> {model_k}")
            break
    if not matched:
        print(f"No match found for: {ckpt_k}, shape: {ckpt_state_dict[ckpt_k].shape}")
        break  # Break loop if further keys are not processed

Skipping: llm.model.lm_head.weight
Skipping: protein_encoder.model.esm.contact_head.regression.weight
Skipping: protein_encoder.model.esm.contact_head.regression.bias
Skipping: protein_encoder.model.lm_head.bias
Skipping: protein_encoder.model.lm_head.dense.weight
Skipping: protein_encoder.model.lm_head.dense.bias
Skipping: protein_encoder.model.lm_head.layer_norm.weight
Skipping: protein_encoder.model.lm_head.layer_norm.bias
Skipping: protein_encoder.model.lm_head.decoder.weight


In [5]:
model.load_state_dict(state_dict=applied_ckpt_state_dict)

<All keys matched successfully>

In [6]:
model.save_pretrained('evolla-base')

[2025-02-12 11:09:02,148] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/envs/pl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/envs/pl/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


# save and load processor

In [3]:
from transformers import EsmTokenizer, LlamaTokenizerFast, EvollaProcessor
import transformers
import importlib
importlib.reload(transformers)
from transformers import EsmTokenizer, LlamaTokenizerFast, EvollaProcessor
protein_tokenizer = EsmTokenizer.from_pretrained("/zhouxibin/workspaces/ProteinQA/Models/SaProt_35M_AF2")
tokenizer = LlamaTokenizerFast.from_pretrained("/zhouxibin/workspaces/ProteinQA/Models/meta-llama_Meta-Llama-3-8B-Instruct")
processor = EvollaProcessor(protein_tokenizer, tokenizer)
processor.save_pretrained("evolla-base")
processor = EvollaProcessor.from_pretrained("evolla-base")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
